In [ ]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
# for visualization self-attention
!pip install -q bertviz~=1.4.0

# Transformer: how do they work internally?

## Transformer in a nutshell

Transformer 由 Encoder 和 Decoder 两部分组成，Encoder 接受 **输入** 然後生成 **向量** 表示這個输入，Decoder 接受 **向量** 然後逐一生成 **输出**。
![transformer in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoder-decoder-1-1024x275.png)

### Encoder

Transformer 由 N 個 Encoder 組成，每個 Encoder 將其輸出送到下一個 Encoder。最終的 Encoder 返回 **輸入** 的 **向量** 表示。為了說明，從現在開始，我們將使用 N=2 的值。

![encoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.png)

### Decoder

同樣地，我們可以有 N 個 Decoder（假設 N=2）。Encoder 生成的 **向量** 表示是所有 Decoder 的輸入；也就是說，一個 Decoder 接收兩個輸入，一個來自前一個 Decoder，另一個來自 Encoder 生成的 **向量** 表示。

![decoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoderNdecoder.png)

## Introduction to Transformer

Google Research 和 Google Brain 的成員最初在 2017 年的論文《[Attention is all you need](https://arxiv.org/abs/1706.03762?context=cs)》中提出了 Transformer。目前是 NLP 中最受歡迎的架構之一。

![Original Transformer Architecture, Source: Attention is all you need](https://www.alexisalulema.com/wp-content/uploads/2022/08/image-695x1024.png)

Transformer 超越了基於 LSTM 和 RNN 的其他現有架構，獲得了更高的評估結果和更快的訓練速度。Transformer 的注意力機制是一種「word-to-word」的操作，它會找出序列中每個詞與其他詞之間的關係，包括詞本身。

在本文中，我將以最簡單的方式解釋 Transformer 的架構和內部運作。

### Input Embedding

Transformer 架構首先是 Input Embedding 層，它將輸入序列轉換為維度為 $\d_{model} = 512$ 的向量。

![Input Embedding](https://www.alexisalulema.com/wp-content/uploads/2022/08/input.embedding.png)

> $\ d_{model} = 512 $ 的值是由架構設計者設定的，這個維度的值可以根據目標進行修改。

Tokenizer 通過將文本正規化來將輸入流轉換為 token；此外，它還會提供一個整數表示（基於現有詞彙），該表示將用於 Embedding 過程。例如：

In [ ]:
from transformers import AutoTokenizer
from pprint import pprint

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

pprint(tokenizer)
# vocab_szie: 30522
# model_max_length: 512
# 0: [PAD]
# 100: [UNK]
# 101: [CLS]
# 102: [SEP]
# 103: [MASK]

pprint(list(tokenizer.vocab.keys())[1000:1010])
# display 10 vocab


In [ ]:
raw_inputs = [
    "I've been waiting for a this course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
  )

pprint(inputs)
# 1. 第二句子長度不夠，所以會被補上 [PAD] token
# 2. Attention mask 會標記出哪些是 padding 的 token

tokenizer.decode(inputs["input_ids"][1])

如果將此模型應用在中文文本上，會發生什麼狀況呢？

In [ ]:
# Multilingual
# model_name = "google-bert/bert-base-multilingual-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# pprint(tokenizer)

raw_inputs = [
    "我門正在學習目前正夯的變形金剛模型！",
    "你們喜歡這個課程嗎？"
]
inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
  )

tokenizer.decode(inputs["input_ids"][1])

接下來，Embedding 從分詞後的向量中獲取數據。對於每個詞，生成一個大小為 $\ d_{model} = 512 $ 的向量。例如，對於表示意思的詞，Embedding 向量應該是相似的：

In [ ]:
from transformers import AutoTokenizer, AutoModel
from pprint import pprint

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a new example sentence with multiple meanings of the word "bank"
text = '''
After stealing money from the bank vault, the bank robber was seen
fishing on the Mississippi river bank.
'''

# Map the token strings to their vocabulary indeces.
text_to_token_id = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# Display the words with their indeces.
# 請注意，這裡的 token id 是對應到 vocab 的 index
# 此時 bank 這個字具有相同的 token id
for id in text_to_token_id["input_ids"][0]:
    pprint('{:<12} {:>6,}'.format(tokenizer.decode(id), id))


In [ ]:
model = AutoModel.from_pretrained(
  model_name,
  output_hidden_states=True,  # Whether the model returns all hidden-states.
  )

# 這個 model 有 12 層的 transformer blocks, 及第一層的 embedding 層, 共 13 層
pprint(model)

In [ ]:
import torch

# Run the text through BERT, and collect all of the hidden states produced from all 12 layers.
with torch.no_grad():
    outputs = model(**text_to_token_id)
    hidden_states = outputs.hidden_states

n_hidden_states = len(hidden_states)
pprint(f"Number of layers: {n_hidden_states} (initial embeddings + 12 BERT layers)")
layer_i = 0

n_batches = len(hidden_states[layer_i])
pprint(f"Number of batches: {n_batches}")
batch_i = 0

n_tokens = len(hidden_states[layer_i][batch_i])
pprint(f"Number of tokens: {n_tokens}")
token_i = 0

n_hidden_units = len(hidden_states[layer_i][batch_i][token_i])
pprint(f"Number of hidden units: {n_hidden_units}")

pprint(f"Number of unique values: {n_hidden_states * n_batches * n_tokens * n_hidden_units}")


這個模型的完整隱藏狀態集，存儲在 hidden_states 物件中，這個物件有四個維度，順序如下：

1. 層數（13 層）
2. 批次數（1 句話）
3. 詞/標記 (Token) 數（我們句子中的 22 個 Token）
4. 隱藏單元/特徵數（768 個特徵）

僅僅為了表示我們的一句話，就需要 219,648 個唯一值！

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()


In [ ]:
# Let’s get rid of the “batches” dimension since we don’t need it.
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

In [ ]:
# Current dimensions: [# layers, # tokens, # features]
# Desired dimensions: [# tokens, # layers, # features]

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

現在，我們該如何處理這些隱藏狀態呢？我們希望為每個標記 (Token) 獲取單獨的向量，但對於我們的每個輸入標記 (Token)，我們有 13 個長度為 768 的單獨向量。

為了獲取單獨的向量，我們需要結合一些層向量，但是哪一層或哪幾層的組合能提供最佳表示呢？

不幸的是，沒有一個簡單的答案，不過讓我們嘗試通過將最後四層相加來創建詞向量，為每個標記 (Token) 生成一個詞向量。

每個向量的長度將是維持長度 768。

In [ ]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 13 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [13 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

pprint(f'Shape is: {len(token_vecs_sum)} x {len(token_vecs_sum[0])}')

為了確認這些向量的值確實是依賴於上下文的，讓我們看看在我們示例句子中「bank」這個詞的不同實例：

「After stealing money from the **bank vault**, the **bank robber** was seen fishing on the Mississippi **river bank**.」

讓我們找出這個示例句子中三個「bank」實例的索引。

In [ ]:
for i, id in enumerate(text_to_token_id["input_ids"][0]):
    pprint(f'{i} {tokenizer.decode(id)}')

# They are at 6, 10, and 19.
# We can try printing out their vectors to compare them.
pprint('First 5 vector values for each instance of "bank".')
pprint('')
pprint(f"bank vault  {token_vecs_sum[6][:5]}")
pprint(f"bank robber {token_vecs_sum[10][:5]}")
pprint(f"river bank  {token_vecs_sum[19][:5]}")


In [ ]:
# We can see that the values differ, but let’s calculate the cosine similarity between the vectors to make a more precise comparison.
# Cosine similarity measures the cosine of the angle between two vectors in a high-dimensional space.
# It is a value between -1 and 1, with 1 indicating that the two vectors are identical and 0 indicating that they are orthogonal (i.e., they have no correlation).
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

pprint('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
pprint('Vector similarity for *different* meanings:  %.2f' % diff_bank)

### Positional Encoding (PE)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/positional.encoding.png)

Embedding 向量為 Transformer 提供了大量有關序列中詞語之間關係的信息。然而，仍然需要信息來指示詞語在序列中的位置，這就是位置編碼過程的用途。

初始序列中的每個詞都必須具有位置編碼（PE）信息，但由於 Transformer 的主要重點是注意力機制，因此這個向量的生成必須簡單。

這項任務的挑戰在於為位置編碼函數的每個輸出向量生成一個維度為 $\ d_{model} = 512 $ 的向量。架構的作者找到了一種巧妙的方法，使用正弦和餘弦值表示位置編碼。

$$ PE_{\text{sin}}(\text{pos}, 2i) = \sin\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right) $$

$$ PE_{\text{cos}}(\text{pos}, 2i+1) = \cos\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right) $$

最後，詞編碼向量應該加到位置編碼向量上，然後將結果輸入到編碼器/解碼器區塊中：

$$ Embedding_{output} = Embedding_{initial} + PE_{word-position} $$

### The encoder

每個解碼器區塊由兩個子層組成：

1. 多頭注意力 (Multi-head attention)
2. 前饋神經網路 (Feedforward Network)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.inside-768x315.png)

在開始解釋這兩個組件之前，有必要先了解自注意力 (self-attention) 機制。

#### Self-attention Mecanism

考慮以下句子：

```
John and Paul wrote several songs when they were inspired.
```

在這個句子中，自注意力機制計算每個詞的表示，並且與句子中其他詞的關係提供了更多關於該詞的信息。例如，「they」這個詞應該與「John」和「Paul」相關，而不是與「songs」相關。




In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "John and Paul wrote several songs when they were inspired."
# Tokenize input text
text_to_token_id = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

model = AutoModel.from_pretrained(
    model_name,
    output_attentions=True, # Whether the model returns attentions weights
    )

# Run the text through BERT
with torch.no_grad():
    outputs = model(**text_to_token_id)
    # Retrieve attention from model outputs
    attention = outputs.attentions

tokens = tokenizer.convert_ids_to_tokens(text_to_token_id['input_ids'][0])  # Convert input ids to token strings
print(tokens)


In [ ]:
from bertviz import (
  model_view,
  head_view,
)

# Visualize the self-attention of the model
# Review the relationship of term 'they' along with 'John' and 'Paul'
head_view(
  attention,
  tokens,
  layer=8,
  heads=[9],
  )

In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show

tokenizer = BertTokenizer.from_pretrained(
  'bert-base-uncased',
  do_lower_case=True)
model = BertModel.from_pretrained(
  'bert-base-uncased',
  output_attentions=True)
show(
  model=model,
  model_type='bert',
  tokenizer=tokenizer,
  sentence_a=text,
  layer=8,
  head=9,
  )